<a href="https://colab.research.google.com/github/mueqee/MBTI-LSTM/blob/main/notebooks/MBTI_LSTM_Training_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MBTI-LSTM. Обучение модели. Google Colab

---

Классификация типов личности MBTI на основе текстов из социальных сетей с использованием LSTM архитектуры.

**Архитектура модели:**
- Embedding (300d) → BiLSTM(128) → BiLSTM(64) → Dropout(0.2) → FC(64, ReLU) → Output(4, Sigmoid)
- 4 бинарных классификатора для дихотомий: I/E, N/S, T/F, J/P

**Ожидаемый результат:** 80-86% accuracy

---

vers. 1.1.1


## 1. GPU

Настройка:
1. Runtime → Change runtime type
2. Hardware accelerator → GPU (T4)
3. Save

Проверка GPU через import torch

In [ ]:
# Проверка GPU
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    !nvidia-smi
else:
    print("⚠️ GPU не обнаружен!")


## 2. Клон репозиторий

In [ ]:
# Клонировать репозиторий
!git clone https://github.com/mueqee/MBTI-LSTM.git
%cd MBTI-LSTM

# Проверить структуру
!ls -la


## 3. Установка зависимостей
через pip install -q -r requirements.txt


In [ ]:
# Установить зависимости
%pip install -q -r requirements.txt
%pip install -q -e .

# Скачать NLTK данные
import nltk
nltk.download('punkt', quiet=True)
nltk.download('punkt_tab', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)

print("✅ Зависимости установлены!")


In [ ]:
%pip install -q "ipykernel==6.17.1" "jupyter-client<8" "notebook<7"

## 4. Загрузка датасета Kaggle
Получить API токен через https://www.kaggle.com/settings -> API -> Create New Token
from google.colab import files


In [ ]:
# 1. Получить API токен: https://www.kaggle.com/settings -> API -> Create New Token
# 2. Загрузить kaggle.json

from google.colab import files
print("Загрузите файл kaggle.json:")
uploaded = files.upload()

# Настроить Kaggle
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Скачать датасет
!kaggle datasets download -d datasnaek/mbti-type
!unzip -q mbti-type.zip -d data/raw/
!rm mbti-type.zip
!mv data/raw/mbti_1.csv data/raw/mbti_dataset.csv

print("Датасет скачан с Kaggle")


## 5. Анализ датасета
Загрузить датасет
Через import pandas as pd

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Загрузить датасет
df = pd.read_csv('data/raw/mbti_dataset.csv')

print(f"Размер датасета: {df.shape}")
print(f"Колонки: {df.columns.tolist()}")
print(f"\n Распределение MBTI типов:")
type_counts = df['type'].value_counts()
print(type_counts)

# Визуализация
plt.figure(figsize=(12, 6))
type_counts.plot(kind='bar', color='steelblue')
plt.title('Распределение MBTI типов в датасете')
plt.xlabel('MBTI тип')
plt.ylabel('Количество')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print(f"\nПример поста (первые 200 символов):")
print(df['posts'].iloc[0][:200] + "...")


## 6. Обновление репозитория

Подтягиваем свежие изменения из гитхаба командой чтобы синхронизировать код перед дальнейшими шагами.


In [ ]:
!git pull

## 7. Поиск гиперпараметров через Optuna

Вместо ручного подбора параметров используем байесовскую оптимизацию.

Установка через pip install -q optuna optuna-dashboard

In [ ]:
# Установка Optuna для поиска гиперпараметров
%pip install -q optuna optuna-dashboard

import optuna
from optuna.trial import Trial
import numpy as np
import json
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

print("✅ Optuna установлен для умного поиска гиперпараметров")


## 8. Импорт модулей
Через import sys


In [ ]:
# Импорт модулей проекта
import sys
sys.path.append('/content/MBTI-LSTM')

from src.data.preprocessor import MBTIPostPreprocessor
from src.data.dataset import create_data_loaders
from src.models.lstm_model import LSTMMBTIClassifier
from src.training.trainer import MBTITrainer
from src.training.metrics import MBTIMetrics

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from google.colab import output
output.enable_custom_widget_manager()

print("✅ Модули проекта импортированы")


## 9. Гиперпараметры для поиска
objective() в Optuna

In [ ]:
def objective(trial: Trial) -> float:
    """
    Целевая функция для Optuna.
    Возвращает отрицательную val_accuracy (минимизируем).
    """
    
    config = {
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 5e-3, log=True),
        'batch_size': trial.suggest_categorical('batch_size', [16, 32, 64]),
        'hidden_dim_1': trial.suggest_categorical('hidden_dim_1', [64, 128, 256]),
        'hidden_dim_2': trial.suggest_categorical('hidden_dim_2', [32, 64, 128]),
        'dropout': trial.suggest_float('dropout', 0.1, 0.4, step=0.05),
        'optimizer': trial.suggest_categorical('optimizer', ['adam', 'adamw', 'rmsprop']),
        'weight_decay': trial.suggest_float('weight_decay', 0.0, 0.001, step=0.0001),
    }
    
    print(f"\nTrial {trial.number}: {config['optimizer']}, LR={config['learning_rate']:.4f}, BS={config['batch_size']}")
    
    try:
        preprocessor = MBTIPostPreprocessor(
            lowercase=True,
            remove_urls=True,
            remove_special_chars=True,
            remove_stopwords=True,
            lemmatize=True
        )
        
        loaders = create_data_loaders(
            df,
            preprocessor=preprocessor,
            max_length=300,
            batch_size=config['batch_size'],
            val_split=0.15,
            test_split=0.15,
            num_workers=2
        )
        train_loader = loaders['train']
        val_loader = loaders['val']
        vocab = loaders['vocabulary']
        
        model = LSTMMBTIClassifier(
            vocab_size=len(vocab),
            embedding_dim=200,
            hidden_dim_1=config['hidden_dim_1'],
            hidden_dim_2=config['hidden_dim_2'],
            dropout=config['dropout']
        )
        
        if config['optimizer'] == 'adam':
            optimizer = torch.optim.Adam(
                model.parameters(),
                lr=config['learning_rate'],
                weight_decay=config['weight_decay']
            )
        elif config['optimizer'] == 'adamw':
            optimizer = torch.optim.AdamW(
                model.parameters(),
                lr=config['learning_rate'],
                weight_decay=config['weight_decay']
            )
        else:
            optimizer = torch.optim.RMSprop(
                model.parameters(),
                lr=config['learning_rate'],
                weight_decay=config['weight_decay']
            )
        
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        trainer = MBTITrainer(
            model=model,
            train_loader=train_loader,
            val_loader=val_loader,
            criterion=nn.BCELoss(),
            optimizer=optimizer,
            device=device,
            checkpoint_dir='tmp_optuna',
            save_best_only=False,
            early_stopping_patience=2
        )
        
        best_val_acc = 0.0
        for epoch in range(1, 6):
            trainer.current_epoch = epoch - 1
            train_metrics = trainer.train_epoch()
            trainer.current_epoch = epoch - 1
            val_metrics = trainer.validate()
            
            train_loss = train_metrics['loss']
            train_acc = train_metrics['overall_accuracy']
            val_loss = val_metrics['loss']
            val_acc = val_metrics['overall_accuracy']
            
            print(
                f"    Эпоха {epoch}: train_loss={train_loss:.4f}, train_acc={train_acc:.4f}, "
                f"val_loss={val_loss:.4f}, val_acc={val_acc:.4f}"
            )
            
            if val_acc > best_val_acc:
                best_val_acc = val_acc
            
            trial.report(val_acc, epoch)
            if trial.should_prune():
                print(f"  ⚠️ Pruning на эпохе {epoch}")
                raise optuna.TrialPruned()
        
        print(f"  ✅ Лучшая val_acc: {best_val_acc:.4f}")
        return -best_val_acc
    
    except Exception as e:
        print(f"  ❌ Ошибка: {e}")
        return 0.0



## 10. Optuna study
Через study = optuna.create_study()


In [ ]:
# Создаём Optuna study
study = optuna.create_study(
    study_name='mbti_lstm_hp_search',
    direction='minimize',
    pruner=optuna.pruners.MedianPruner(
        n_startup_trials=3,
        n_warmup_steps=2,
        interval_steps=1
    )
)

print("="*60)
print("ПОИСК ГИПЕРПАРАМЕТРОВ ЧЕРЕЗ OPTUNA")
print("="*60)
print("Количество попыток: 15 ")
print("Эпох на попытку: 5")
print("Время: ~30-40 минут")
print("="*60)


## 11. Оптимизация
Через study.optimize()


In [ ]:
# Запуск оптимизации
study.optimize(
    objective,
    n_trials=15,  
    gc_after_trial=True,
    show_progress_bar=True
)


## 12. Вывод и визуализация результатов поиска


In [ ]:
# Анализ результатов поиска
print("="*60)
print("РЕЗУЛЬТАТЫ ПОИСКА ГИПЕРПАРАМЕТРОВ")
print("="*60)
print(f"\\nЛучшая accuracy: {-study.best_value:.4f}")
print("\\nЛучшие параметры:")
for key, value in study.best_params.items():
    print(f"  {key}: {value}")

# Сохраняем лучшую конфигурацию
best_config = {
    'accuracy': -study.best_value,
    'params': study.best_params,
    'n_trials': len(study.trials),
    'timestamp': datetime.now().isoformat()
}

config_filename = f'best_config_{datetime.now():%Y%m%d_%H%M}.json'
with open(config_filename, 'w') as f:
    json.dump(best_config, f, indent=2)

print(f"\\n✅ Конфигурация сохранена: {config_filename}")


In [ ]:
# Визуализация результатов поиска
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. История оптимизации
trials_df = study.trials_dataframe()
axes[0,0].plot(trials_df.index, -trials_df['value'], 'b-', alpha=0.5, label='Trial accuracy')
axes[0,0].plot(trials_df.index, (-trials_df['value']).cummax(), 'r-', linewidth=2, label='Best so far')
axes[0,0].set_xlabel('Trial')
axes[0,0].set_ylabel('Validation Accuracy')
axes[0,0].set_title('История оптимизации')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# 2. Важность параметров
importances = optuna.importance.get_param_importances(study)
params = list(importances.keys())
values = list(importances.values())
axes[0,1].barh(params, values)
axes[0,1].set_xlabel('Важность')
axes[0,1].set_title('Важность гиперпараметров')
axes[0,1].grid(True, alpha=0.3)

# 3. Learning Rate vs Accuracy
if 'params_learning_rate' in trials_df.columns:
    axes[1,0].scatter(trials_df['params_learning_rate'], -trials_df['value'], alpha=0.6)
    axes[1,0].set_xlabel('Learning Rate')
    axes[1,0].set_ylabel('Accuracy')
    axes[1,0].set_title('Learning Rate vs Accuracy')
    axes[1,0].grid(True, alpha=0.3)

# 4. Optimizer comparison
if 'params_optimizer' in trials_df.columns:
    optimizer_stats = trials_df.groupby('params_optimizer')['value'].agg(['mean', 'std'])
    optimizer_stats['mean'] = -optimizer_stats['mean']
    optimizer_stats['std'] = optimizer_stats['std']

    axes[1,1].bar(optimizer_stats.index, optimizer_stats['mean'],
                  yerr=optimizer_stats['std'], capsize=5)
    axes[1,1].set_ylabel('Accuracy')
    axes[1,1].set_title('Сравнение оптимизаторов')
    axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\\n📊 Визуализация результатов поиска готова!")


## 13. Проверка лучших параметров

Через best_params = study.best_params


In [ ]:
# Используем лучшие параметры для финального обучения
best_params = study.best_params

print("="*60)
print("ФИНАЛЬНОЕ ОБУЧЕНИЕ С ЛУЧШИМИ ПАРАМЕТРАМИ")
print("="*60)
print("Параметры:")
for k, v in best_params.items():
    print(f"  {k}: {v}")

preprocessor = MBTIPostPreprocessor(
    lowercase=True,
    remove_urls=True,
    remove_special_chars=True,
    remove_stopwords=True,
    lemmatize=True
)

loaders = create_data_loaders(
    df,
    preprocessor=preprocessor,
    max_length=500,
    batch_size=best_params['batch_size'],
    val_split=0.15,
    test_split=0.15,
    num_workers=2
)
train_loader = loaders['train']
val_loader = loaders['val']
test_loader = loaders['test']
vocab = loaders['vocabulary']

final_model = LSTMMBTIClassifier(
    vocab_size=len(vocab),
    embedding_dim=300,
    hidden_dim_1=best_params['hidden_dim_1'],
    hidden_dim_2=best_params['hidden_dim_2'],
    dropout=best_params['dropout']
)

if best_params['optimizer'] == 'adam':
    final_optimizer = torch.optim.Adam(
        final_model.parameters(),
        lr=best_params['learning_rate'],
        weight_decay=best_params['weight_decay']
    )
elif best_params['optimizer'] == 'adamw':
    final_optimizer = torch.optim.AdamW(
        final_model.parameters(),
        lr=best_params['learning_rate'],
        weight_decay=best_params['weight_decay']
    )
else:
    final_optimizer = torch.optim.RMSprop(
        final_model.parameters(),
        lr=best_params['learning_rate'],
        weight_decay=best_params['weight_decay']
    )

num_params = sum(p.numel() for p in final_model.parameters() if p.requires_grad)
print()
print(f"Модель создана: {num_params:,} параметров")
print(f"Оптимизатор: {best_params['optimizer']}")
print(f"Learning rate: {best_params['learning_rate']}")



## 14. Финальное обучение

In [ ]:
# Финальное обучение
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

final_trainer = MBTITrainer(
    model=final_model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=nn.BCELoss(),
    optimizer=final_optimizer,
    device=device,
    checkpoint_dir='checkpoints/optimized',
    save_best_only=True,
    early_stopping_patience=5
)

NUM_EPOCHS = 20

train_history = {'loss': [], 'acc': []}
val_history = {'loss': [], 'acc': []}
print(f"Начинаем финальное обучение на {NUM_EPOCHS} эпох...")
print("="*60)

best_val_acc = 0.0
BEST_MODEL_PATH = 'best_optimized_model.pth'

for epoch in range(1, NUM_EPOCHS + 1):
    final_trainer.current_epoch = epoch - 1
    train_metrics = final_trainer.train_epoch()
    final_trainer.current_epoch = epoch - 1
    val_metrics = final_trainer.validate()

    train_loss = train_metrics['loss']
    train_acc = train_metrics['overall_accuracy']
    val_loss = val_metrics['loss']
    val_acc = val_metrics['overall_accuracy']

    train_history['loss'].append(train_loss)
    train_history['acc'].append(train_acc)
    val_history['loss'].append(val_loss)
    val_history['acc'].append(val_acc)

    dichotomy_names = ['I/E', 'N/S', 'T/F', 'J/P']
    val_dichotomies = {name: val_metrics[f'{name}_accuracy'] for name in dichotomy_names}

    print(
        f"Epoch {epoch:2d}/{NUM_EPOCHS}: "
        f"Train Loss={train_loss:.4f}, Acc={train_acc:.4f} | "
        f"Val Loss={val_loss:.4f}, Acc={val_acc:.4f}"
    )

    if epoch % 5 == 0:
        print(
            "  Дихотомии: " + ", ".join(
                [f"{name}={val_dichotomies[name]:.3f}" for name in dichotomy_names]
            )
        )

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save({
            'epoch': epoch,
            'model_state_dict': final_model.state_dict(),
            'optimizer_state_dict': final_optimizer.state_dict(),
            'val_acc': val_acc,
            'config': best_params
        }, BEST_MODEL_PATH)
        print(f"  ✅ Новая лучшая модель! (Val Acc: {val_acc:.4f})")

print("="*60)
print(f"Обучение завершено! Лучшая accuracy: {best_val_acc:.4f}")



## 15. Тестирование на тестовой выборке
Чрез test_loss, test_acc, test_dichotomies = final_trainer.validate(test_loader)


In [ ]:
# Тестирование на тестовой выборке
criterion = nn.BCELoss()

def evaluate_loader(loader):
    final_model.eval()
    metrics = MBTIMetrics(device=device)
    total_loss = 0.0
    total_samples = 0
    with torch.no_grad():
        for batch in loader:
            input_ids = batch['input_ids'].to(device)
            lengths = batch['length'].to(device)
            labels = batch['labels'].to(device)

            outputs = final_model(input_ids, lengths)
            loss = criterion(outputs, labels)

            batch_size = input_ids.size(0)
            total_loss += loss.item() * batch_size
            total_samples += batch_size

            metrics.update((outputs > 0.5).float(), labels, outputs)

    results = metrics.compute()
    results['loss'] = total_loss / max(total_samples, 1)
    return results

test_metrics = evaluate_loader(test_loader)

test_loss = test_metrics['loss']
test_acc = test_metrics['overall_accuracy']

dichotomy_names = ['I/E', 'N/S', 'T/F', 'J/P']
test_dichotomies = {name: test_metrics[f'{name}_accuracy'] for name in dichotomy_names}

print("="*60)
print("РЕЗУЛЬТАТЫ НА ТЕСТОВОЙ ВЫБОРКЕ")
print("="*60)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print("Точность по дихотомиям:")
for key, value in test_dichotomies.items():
    print(f"  {key}: {value:.4f}")

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(train_history['loss'], label='Train Loss')
axes[0].plot(val_history['loss'], label='Val Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training and Validation Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(train_history['acc'], label='Train Acc')
axes[1].plot(val_history['acc'], label='Val Acc')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].set_title('Training and Validation Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()



## 16. Выгрузка в Google Drive


In [ ]:
# Сохранение результатов в Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Создаём папку для результатов
import os
import shutil
save_dir = '/content/drive/MyDrive/MBTI-LSTM-Optimized-Results'
os.makedirs(save_dir, exist_ok=True)

# Сохраняем файлы
shutil.copy('best_optimized_model.pth', save_dir)
shutil.copy(config_filename, save_dir)

# Сохраняем результаты
results = {
    'best_hyperparams': best_params,
    'best_val_acc': float(best_val_acc),
    'test_acc': float(test_acc),
    'test_dichotomies': {k: float(v) for k, v in test_dichotomies.items()},
    'num_trials': len(study.trials),
    'training_epochs': NUM_EPOCHS,
    'timestamp': datetime.now().isoformat()
}

results_filename = f'optimized_results_{datetime.now():%Y%m%d_%H%M}.json'
with open(results_filename, 'w') as f:
    json.dump(results, f, indent=2)

shutil.copy(results_filename, save_dir)

print(f"\\n✅ Все результаты сохранены в Google Drive: {save_dir}")
print(f"✅ Модель: best_optimized_model.pth")
print(f"✅ Конфигурация: {config_filename}")
print(f"✅ Результаты: {results_filename}")

print("\\n" + "="*60)
print("ИТОГОВЫЕ РЕЗУЛЬТАТЫ")
print("="*60)
print(f"Лучшая Val Accuracy: {best_val_acc:.4f}")
print(f"Test Accuracy: {test_acc:.4f}")
print(f"\\nУлучшение по сравнению с базовой моделью:")
print(f"  Базовая модель: ~71%")
print(f"  Оптимизированная: {test_acc:.1%}")
print(f"  Прирост: +{test_acc*100-71:.1f}%")

if test_acc > 0.75:
    print("\\n🎉 Отличный результат! Модель готова к production!")
elif test_acc > 0.72:
    print("\\n✅ Хороший результат! Можно добавить attention для улучшения.")
else:
    print("\\n⚠️ Результат можно улучшить. Попробуйте:")
    print("  - Увеличить количество эпох до 50")
    print("  - Добавить attention механизм")
    print("  - Использовать pretrained embeddings")


## 17. Или удалить и клон снова:


In [ ]:

# Или удалить и клон снова:
!rm -rf MBTI-LSTM
!git clone https://github.com/mueqee/MBTI-LSTM.git
%cd MBTI-LSTM

## 18. Быстрый тест на 2 эпохи
Через !python scripts/train.py \


In [ ]:
!python scripts/train.py \
    --data_path data/raw/mbti_dataset.csv \
    --num_epochs 2 \
    --batch_size 32 \
    --checkpoint_dir checkpoints/quick_test

print("\n✅ Тест пройден! Модель обучается корректно")


## 19. Обучение на 50 эпох

Запуск тренировочного скрипта из CLI
 scripts/train.py, чтобы проверить полноценный пайплайн обучения из репозитория

In [ ]:
print("Запуск финального обучения на 50 эпох...\n")

!python scripts/train.py \
    --data_path data/raw/mbti_dataset.csv \
    --num_epochs 50 \
    --batch_size 64 \
    --learning_rate 0.001 \
    --optimizer adam \
    --hidden_dim_1 128 \
    --hidden_dim_2 64 \
    --dropout 0.2 \
    --max_length 750 \
    --balance_classes \
    --num_workers 4 \
    --early_stopping_patience 7 \
    --checkpoint_dir checkpoints/final_model

print("\n✅ Обучение завершено!")


## 20. Сохранение результатов
Описание: from google.colab import drive


In [ ]:
# Сохранить в Google Drive
from google.colab import drive
drive.mount('/content/drive')

!mkdir -p '/content/drive/MyDrive/MBTI-LSTM-Results'
!cp -r checkpoints/final_model '/content/drive/MyDrive/MBTI-LSTM-Results/'
!cp -r checkpoints/quick_test '/content/drive/MyDrive/MBTI-LSTM-Results/'

print("✅ Результаты сохранены в Google Drive!")
print("Путь: /content/drive/MyDrive/MBTI-LSTM-Results/")
